In [ ]:
#引包，dct特征用opencv中的方法来提取。
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import cv2
import time
from __future__ import division

In [ ]:
#读取每张图片的GMM-8
ueas = np.load('ueas.npz')
Us = ueas['U']
Es = ueas['E']
As = ueas['A']

In [ ]:
#各类图像的GMM整合（第二层GMM）
def gaussion_prob(data_U,U,E,th=5e-4):#多数据多高斯同时求概率
    #input_dim:  data_U-(N,dim)  U-(m,dim)  E-(m,dim,dim)
    #output_dim:  prob-(m,N)
    N,dim = data_U.shape
    m = U.shape[0]
    E_d = np.eye(dim)*th
    prob = np.zeros((m,N))
    inv_Es = np.zeros((m,dim,dim))
    for i in range(m):
        det_E = np.linalg.det(E[i]+E_d)
        inv_E = np.linalg.inv(E[i]+E_d)
        inv_Es[i] = inv_E
        xishu = 1 / (det_E ** 0.5 * (2.0 * np.pi) ** (dim/2))
        X_U = data_U - U[i]
        zhishu = np.sum(np.dot(X_U,inv_E) * X_U,axis=1) *(-0.5)
        prob[i] = xishu * np.exp(zhishu)
    return prob,inv_Es

def trace(data_E,iE):
    #input_dim:  data_E-(N,dim,dim)  iE-(m,dim,dim)
    #output_dim: exp_tr-(m,N)
    N,dim,_ = data_E.shape
    m = iE.shape[0]
    tr = np.zeros((m,N))
    for i in range(dim):
        tr += np.dot(iE[:,i,:],data_E[:,:,i].T)#存疑
    exp_tr = np.exp(-0.5*tr)
    return exp_tr

def H(data_PI,PI,prob,trace):
    #input_dim:  data_PI-(N,)  PI-(m,)  prob-(m,N)  trace(m,N)
    #output_dim:  h-(m,N)
    mul = prob * trace
    up = np.power(mul,data_PI) * PI[:,np.newaxis]
    down = np.sum(up,axis=0)
    h = up/down
    return h

def check_diff(x,y,ma,me):
    check_max = np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))
    check_mean = np.mean(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))
    #print 'max',check_max,'\nmean',check_mean
    if check_max < ma and check_mean < me:
        return True
    return False
    
def GMM2(data_U,data_E,data_PI,m):
    #初始化
    N,dim = data_U.shape
    randlist = np.random.choice(N, m, replace=False)
    U = np.copy(data_U[randlist])
    E = np.copy(data_E[randlist])
    PI = np.copy(data_PI[randlist])
    oldU = np.copy(U)
    
    while True:
        #（E步）
        prob,iE = gaussion_prob(data_U,U,E,5e-4)
        tra = trace(data_E,iE)
        h = H(data_PI,PI,prob,tra)
        #（M步）
        PI = np.sum(h,axis=1) / N
        pre_w = np.nan_to_num(h * data_PI)
        w = pre_w/np.sum(pre_w,axis=1,keepdims=True)
        U = np.dot(w,data_U)
        for i in range(m):
            t = data_U - U[i]
            E1 = np.sum(w[i,:,np.newaxis,np.newaxis] * data_E,axis=0)
            E2 = np.dot(t.T*w[i],t)
            E[i] = E1 + E2
        
        #收敛判断
        U = np.nan_to_num(U)
        E = np.nan_to_num(E)
        PI = np.nan_to_num(PI)
        if check_diff(U,oldU,1,0.01):
            break;
        oldU = np.copy(U)
        
    return (U,E,PI)

In [ ]:
#对标签进行选取，只要出现10次以上的（共有192个），用label_sets来保存各个标签的图片编号，并保存每个标签出现的次数。
train_label = np.loadtxt('train.csv',delimiter=',')
train_label_count = train_label.sum(0)
label_index = np.arange(260)[train_label_count > 10]
train_label_samp = train_label[:,label_index]
label_sets = []
for i in range(train_label_samp.shape[1]):
    label_sets.append(np.arange(4500)[train_label_samp[:,i] == 1])
np.save('label_times',train_label_count[label_index])

In [ ]:
#生成最终的模型
models = []
tic = time.time()
for i in range(len(label_sets)):
    U = np.nan_to_num(Us[label_sets[i]].reshape(8*label_sets[i].shape[0],63))
    E = np.nan_to_num(Es[label_sets[i]].reshape(8*label_sets[i].shape[0],63,63))
    A = np.nan_to_num(As[label_sets[i]].reshape(8*label_sets[i].shape[0]))
    models.append(GMM2(U,E,A,64))
    print i,':',time.time() - tic

In [ ]:
#保存最终模型
def model_change_dump(model_to_change,dump_name):
    n = len(model_to_change)
    U_dump = np.zeros((n,64,63))
    E_dump = np.zeros((n,64,63,63))
    A_dump = np.zeros((n,64))
    for i in range(n):
        U_dump[i] = model_to_change[i][0]
        E_dump[i] = model_to_change[i][1]
        A_dump[i] = model_to_change[i][2]
    np.savez_compressed(dump_name,U=U_dump,E=E_dump,A=A_dump)
model_change_dump(models3,'models')

In [ ]:
#载入标签字符对应词典,截取词典的一部分,保存为新的词典
cor_dict = np.loadtxt('corel5k_words.txt',dtype=np.str)
cor_dict_samp = cor_dict[label_index]
np.savetxt('new_words.txt',cor_dict_samp,fmt='%s',delimiter='')